<a href="https://colab.research.google.com/github/PhilipOe1612340/CVIS_DeepLearning/blob/master/exercises_01/ex1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!git clone https://github.com/PhilipOe1612340/CVIS_DeepLearning.git

fatal: destination path 'CVIS_DeepLearning' already exists and is not an empty directory.


In [0]:
import glob
# find images in exercise 1
images = glob.glob("CVIS_DeepLearning/exercises_01/**/*.jpg", recursive=True)